# Introduction

Writing a Numpy extension was undertaken for several purposes including: speeding up the simple python implementation, understanding Numpy internals better (as I use this in my day job), and simply learning. This notebook documents a number of things to be aware of that I haven't been able to find in a concise location or tutorial

## Start with setup.py

Unless you are going to do a drop-dead simple implementation including supporting a single Numpy dtype, the very first thing you'll want to do is to conform to the Numpy setup conventions. Admittedly, I simply copied the setup.py from a clone of the Numpy repository and sliced and stitched to satisfy my needs. The germain portions really are defining the extension configurations as seen in the following snippet.

In [ ]:
def configuration(parent_package='', top_path=None):
    from numpy.distutils.misc_util import Configuration

    config = Configuration('redblackgraph',
                           parent_package,
                           top_path)
    config.add_extension('rb_multiarray',
                         [
                             'redblackgraph/core/src/multiarray/rbg_math.h.src',
                             'redblackgraph/core/src/multiarray/rbg_math.c.src',
                             'redblackgraph/core/src/multiarray/redblack.c.src',
                             'redblackgraph/core/src/multiarray/relational_composition.c.src',
                             'redblackgraph/core/src/multiarray/warshall.c.src'
                         ],
                         include_dirs=['redblackgraph/core/src/multiarray'])


This configuration function is then set as the `configuration` parameter of the metadata passed into `numpy.distutils.core.setup`.

## conv_template.py

Please note the source designations of `filename.c.src` and `filename.h.src`. These are handled specially by Numpy's distutils extensions. Run `python setup.py build -v` and observe the following output:

```
running build
...
build_src
building extension "redblackgraph.rb_multiarray" sources
...
creating build/src.macosx-10.6-intel-3.6/redblackgraph/core/src/multiarray
...
conv_template:> build/src.macosx-10.6-intel-3.6/redblackgraph/core/src/multiarray/redblack.c [1]
...
running build_ext
...
creating build/temp.macosx-10.6-intel-3.6/build/src.macosx-10.6-intel-3.6/redblackgraph/core/src/multiarray
...
clang: build/src.macosx-10.6-intel-3.6/redblackgraph/core/src/multiarray/redblack.c
...
creating build/lib.macosx-10.6-intel-3.6/redblackgraph
/usr/bin/clang ... -o build/lib.macosx-10.6-intel-3.6/redblackgraph/rb_multiarray.cpython-36m-darwin.so
```

The line marked **\[1\]** shows a numpy utility, `numpy.distutils.conv_template` being invoked to convert the `redblack.c.src` file into `redblack.c` in the `build/src` directory tree. 

conv_template is a Numpy utility that provides rudimentary templating support in the spirit of C++ templates. Aside from the file's docstring, there really isn't any other documentation. 

Bottom line is that any source files designated in `config.add_extension` ending in `.src` will be run through the template expansion prior to the compilation stage. The generate files will be used (from the `build/src` directory) rather than the source files specified in `config.add_extension`.